In [ ]:
pip install umap-learn

In [1]:
!pip install git+https://huggingface.co/ctheodoris/Geneformer

  Cloning https://huggingface.co/ctheodoris/Geneformer to /tmp/pip-req-build-xu67ghvq
  Running command git clone --filter=blob:none --quiet https://huggingface.co/ctheodoris/Geneformer /tmp/pip-req-build-xu67ghvq
  Resolved https://huggingface.co/ctheodoris/Geneformer to commit 5a4383248d230ea384b289f4378f255573ad5d4b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 78.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup

In [2]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
import json

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
def trunc(text):
    return text[:-4]

In [6]:
ase_rows = pd.read_feather("/content/drive/MyDrive/ase_data/ase_rows_only_with_p_localized.feather")

In [7]:
output_list = pd.read_csv('/content/drive/MyDrive/ase_data/TCGA_localized_PCa_subtypes.txt', sep= "\t")

In [8]:
output_list = output_list.rename(columns={"Unnamed: 0": "caseID"})

In [9]:
file = open("/content/drive/MyDrive/ase_data/ASE_localized_recurr10perc.txt", "r")
content = file.read()
file.close()

ase_localized_recurr10perc = content.split("\n")
ase_localized_recurr10perc.pop()

''

In [10]:
import pickle
with open("/content/drive/MyDrive/ase_data/geneformer_token_dictionary.pkl", "rb") as f:
  gene_token_dict = pickle.load(f)

# Assuming 'ase_rows' is your DataFrame and 'genes' is a Series from this DataFrame
# Your 'parse' function
def parse(gene):
    return gene.split("_")[1].split(".")[0]

# Apply the 'parse' function to each gene in the 'genes' column of 'ase_rows' to create 'ensembl_genes'
ase_rows['ensembl_genes'] = ase_rows['gene'].apply(parse)

# Initialize an empty set to hold genes not covered by the gene_token_dict
excluded_genes = set()

# Check each gene in 'ensembl_genes' against the gene_token_dict to find excluded genes
for gene in set(ase_rows['ensembl_genes']):
    if gene not in gene_token_dict:
        excluded_genes.add(gene)

# Filter 'ase_rows' to exclude rows with genes that are not covered by gene_token_dict
ase_rows_filtered = ase_rows[~ase_rows['ensembl_genes'].isin(excluded_genes)]

In [11]:
ase_rows

,gene,snp,mutect_ref_n,arc_ref_n,mutect_total_n,arc_total_n,foldc_adjust,caseID,dna_ref_frac,rna_ref_frac,...,caseid_snp,genename,caseid_gene,ase,ase_p,FDR,foldc_adjust_new,mae,mae_strong,ensembl_genes
0,ZYX_ENSG00000159840.14,chr7_143382967_A_T,41,119.0,92,250.0,1.127720,TCGA-ZG-A9NI-01A,0.445652,0.476000,...,TCGA-ZG-A9NI-01A_chr7_143382967_A_T,ZYX,TCGA-ZG-A9NI-01A_ZYX,0,0.340468,0.679091,1.129678,0,0,ENSG00000159840
1,ZWINT_ENSG00000122952.15,chr10_56358869_T_C,52,22.0,118,35.0,2.111111,TCGA-ZG-A9NI-01A,0.440678,0.628571,...,TCGA-ZG-A9NI-01A_chr10_56358869_T_C,ZWINT,TCGA-ZG-A9NI-01A_ZWINT,0,0.027331,0.160123,2.144539,0,0,ENSG00000122952
2,ZRANB2_ENSG00000132485.11,chr1_71070891_G_C,30,15.0,62,28.0,1.223437,TCGA-ZG-A9NI-01A,0.483871,0.535714,...,TCGA-ZG-A9NI-01A_chr1_71070891_G_C,ZRANB2,TCGA-ZG-A9NI-01A_ZRANB2,0,0.706088,0.949954,1.230258,0,0,ENSG00000132485
3,ZNFX1_ENSG00000124201.13,chr20_49248972_G_A,24,5.0,56,11.0,1.122449,TCGA-ZG-A9NI-01A,0.428571,0.454545,...,TCGA-ZG-A9NI-01A_chr20_49248972_G_A,ZNFX1,TCGA-ZG-A9NI-01A_ZNFX1,0,1.000000,1.000000,1.110871,0,0,ENSG00000124201
4,ZNF84_ENSG00000198040.9,chr12_133041405_G_A,17,8.0,45,15.0,1.845714,TCGA-ZG-A9NI-01A,0.377778,0.533333,...,TCGA-ZG-A9NI-01A_chr12_133041405_G_A,ZNF84,TCGA-ZG-A9NI-01A_ZNF84,0,0.286361,0.634208,1.879898,0,0,ENSG00000198040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493003,AARS_ENSG00000090861.14,chr16_70253274_A_G,17,81.0,31,146.0,1.030971,TCGA-2A-A8VO-01A,0.548387,0.554795,...,TCGA-2A-A8VO-01A_chr16_70253274_A_G,AARS,TCGA-2A-A8VO-01A_AARS,0,0.933780,1.000000,1.026190,0,0,ENSG00000090861
1493004,AAK1_ENSG00000115977.17,chr2_69514669_C_T,20,6.0,33,8.0,1.712195,TCGA-2A-A8VO-01A,0.606061,0.750000,...,TCGA-2A-A8VO-01A_chr2_69514669_C_T,AAK1,TCGA-2A-A8VO-01A_AAK1,0,0.492619,0.827367,1.946546,0,0,ENSG00000115977
1493005,AACS_ENSG00000081760.15,chr12_125076605_A_G,31,7.0,67,14.0,1.158730,TCGA-2A-A8VO-01A,0.462687,0.500000,...,TCGA-2A-A8VO-01A_chr12_125076605_A_G,AACS,TCGA-2A-A8VO-01A_AACS,0,0.795309,0.971567,1.160942,0,0,ENSG00000081760
1493006,A4GALT_ENSG00000128274.14,chr22_42692965_C_T,22,21.0,48,57.0,0.693760,TCGA-2A-A8VO-01A,0.458333,0.368421,...,TCGA-2A-A8VO-01A_chr22_42692965_C_T,A4GALT,TCGA-2A-A8VO-01A_A4GALT,0,0.185683,0.510798,0.689941,0,0,ENSG00000128274


In [12]:
ase_rows_filtered

,gene,snp,mutect_ref_n,arc_ref_n,mutect_total_n,arc_total_n,foldc_adjust,caseID,dna_ref_frac,rna_ref_frac,...,caseid_snp,genename,caseid_gene,ase,ase_p,FDR,foldc_adjust_new,mae,mae_strong,ensembl_genes
0,ZYX_ENSG00000159840.14,chr7_143382967_A_T,41,119.0,92,250.0,1.127720,TCGA-ZG-A9NI-01A,0.445652,0.476000,...,TCGA-ZG-A9NI-01A_chr7_143382967_A_T,ZYX,TCGA-ZG-A9NI-01A_ZYX,0,0.340468,0.679091,1.129678,0,0,ENSG00000159840
1,ZWINT_ENSG00000122952.15,chr10_56358869_T_C,52,22.0,118,35.0,2.111111,TCGA-ZG-A9NI-01A,0.440678,0.628571,...,TCGA-ZG-A9NI-01A_chr10_56358869_T_C,ZWINT,TCGA-ZG-A9NI-01A_ZWINT,0,0.027331,0.160123,2.144539,0,0,ENSG00000122952
2,ZRANB2_ENSG00000132485.11,chr1_71070891_G_C,30,15.0,62,28.0,1.223437,TCGA-ZG-A9NI-01A,0.483871,0.535714,...,TCGA-ZG-A9NI-01A_chr1_71070891_G_C,ZRANB2,TCGA-ZG-A9NI-01A_ZRANB2,0,0.706088,0.949954,1.230258,0,0,ENSG00000132485
3,ZNFX1_ENSG00000124201.13,chr20_49248972_G_A,24,5.0,56,11.0,1.122449,TCGA-ZG-A9NI-01A,0.428571,0.454545,...,TCGA-ZG-A9NI-01A_chr20_49248972_G_A,ZNFX1,TCGA-ZG-A9NI-01A_ZNFX1,0,1.000000,1.000000,1.110871,0,0,ENSG00000124201
4,ZNF84_ENSG00000198040.9,chr12_133041405_G_A,17,8.0,45,15.0,1.845714,TCGA-ZG-A9NI-01A,0.377778,0.533333,...,TCGA-ZG-A9NI-01A_chr12_133041405_G_A,ZNF84,TCGA-ZG-A9NI-01A_ZNF84,0,0.286361,0.634208,1.879898,0,0,ENSG00000198040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493003,AARS_ENSG00000090861.14,chr16_70253274_A_G,17,81.0,31,146.0,1.030971,TCGA-2A-A8VO-01A,0.548387,0.554795,...,TCGA-2A-A8VO-01A_chr16_70253274_A_G,AARS,TCGA-2A-A8VO-01A_AARS,0,0.933780,1.000000,1.026190,0,0,ENSG00000090861
1493004,AAK1_ENSG00000115977.17,chr2_69514669_C_T,20,6.0,33,8.0,1.712195,TCGA-2A-A8VO-01A,0.606061,0.750000,...,TCGA-2A-A8VO-01A_chr2_69514669_C_T,AAK1,TCGA-2A-A8VO-01A_AAK1,0,0.492619,0.827367,1.946546,0,0,ENSG00000115977
1493005,AACS_ENSG00000081760.15,chr12_125076605_A_G,31,7.0,67,14.0,1.158730,TCGA-2A-A8VO-01A,0.462687,0.500000,...,TCGA-2A-A8VO-01A_chr12_125076605_A_G,AACS,TCGA-2A-A8VO-01A_AACS,0,0.795309,0.971567,1.160942,0,0,ENSG00000081760
1493006,A4GALT_ENSG00000128274.14,chr22_42692965_C_T,22,21.0,48,57.0,0.693760,TCGA-2A-A8VO-01A,0.458333,0.368421,...,TCGA-2A-A8VO-01A_chr22_42692965_C_T,A4GALT,TCGA-2A-A8VO-01A_A4GALT,0,0.185683,0.510798,0.689941,0,0,ENSG00000128274


In [13]:
ase_rows_filtered['caseID'] = ase_rows_filtered['caseID'].apply(trunc)
ase_rows_filtered = ase_rows_filtered[['caseID', 'genename', 'foldc_adjust_new', 'ensembl_genes']]
ase_rows_filtered

<ipython-input-13-5c6afd4556e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ase_rows_filtered['caseID'] = ase_rows_filtered['caseID'].apply(trunc)


,caseID,genename,foldc_adjust_new,ensembl_genes
0,TCGA-ZG-A9NI,ZYX,1.129678,ENSG00000159840
1,TCGA-ZG-A9NI,ZWINT,2.144539,ENSG00000122952
2,TCGA-ZG-A9NI,ZRANB2,1.230258,ENSG00000132485
3,TCGA-ZG-A9NI,ZNFX1,1.110871,ENSG00000124201
4,TCGA-ZG-A9NI,ZNF84,1.879898,ENSG00000198040
...,...,...,...,...
1493003,TCGA-2A-A8VO,AARS,1.026190,ENSG00000090861
1493004,TCGA-2A-A8VO,AAK1,1.946546,ENSG00000115977
1493005,TCGA-2A-A8VO,AACS,1.160942,ENSG00000081760
1493006,TCGA-2A-A8VO,A4GALT,0.689941,ENSG00000128274


In [14]:
# Convert boolean columns to integers (True to 1, False to 0)
binary_columns = ['SPOP', 'FOXA1', 'MSI', 'TMPRSS2--ERG fusion', 'PTEN deletion']
output_list[binary_columns] = output_list[binary_columns].astype(int)

# Extract binary vectors per row (caseID)
binary_vectors = output_list[binary_columns].values.tolist()

# Optionally, create a dictionary with caseID as keys and binary vectors as values
caseID_vectors = dict(zip(output_list['caseID'], binary_vectors))

In [15]:
caseID_vectors

{'TCGA-2A-A8VO': [0, 0, 0, 0, 0],
 'TCGA-2A-A8VT': [0, 0, 0, 1, 0],
 'TCGA-2A-A8VV': [0, 0, 0, 1, 0],
 'TCGA-2A-A8VX': [0, 0, 0, 0, 0],
 'TCGA-2A-A8W1': [0, 0, 0, 0, 0],
 'TCGA-2A-A8W3': [0, 0, 0, 0, 0],
 'TCGA-2A-AAYF': [0, 0, 0, 0, 0],
 'TCGA-2A-AAYO': [0, 0, 0, 0, 0],
 'TCGA-2A-AAYU': [0, 0, 0, 1, 1],
 'TCGA-4L-AA1F': [0, 0, 0, 1, 0],
 'TCGA-CH-5737': [0, 0, 0, 0, 1],
 'TCGA-CH-5738': [0, 0, 0, 0, 0],
 'TCGA-CH-5739': [0, 0, 0, 1, 0],
 'TCGA-CH-5740': [0, 0, 0, 1, 1],
 'TCGA-CH-5741': [0, 0, 0, 1, 1],
 'TCGA-CH-5743': [0, 0, 0, 1, 0],
 'TCGA-CH-5744': [0, 0, 0, 1, 1],
 'TCGA-CH-5745': [0, 0, 0, 0, 0],
 'TCGA-CH-5746': [0, 0, 0, 1, 0],
 'TCGA-CH-5748': [0, 0, 0, 0, 0],
 'TCGA-CH-5750': [0, 0, 1, 0, 0],
 'TCGA-CH-5751': [0, 0, 0, 0, 0],
 'TCGA-CH-5752': [0, 0, 0, 1, 0],
 'TCGA-CH-5753': [0, 0, 0, 0, 0],
 'TCGA-CH-5754': [0, 0, 0, 1, 0],
 'TCGA-CH-5761': [0, 0, 0, 0, 0],
 'TCGA-CH-5762': [0, 0, 0, 0, 0],
 'TCGA-CH-5763': [0, 0, 0, 0, 0],
 'TCGA-CH-5764': [0, 0, 0, 1, 0],
 'TCGA-CH-5765

In [16]:
# Group by 'caseID' and aggregate 'genename' and 'fold_c_value' into lists
aggregated_ase_rows = ase_rows_filtered.groupby('caseID').agg({
    'ensembl_genes': lambda x: list(x),
    'foldc_adjust_new': lambda x: list(x)
}).reset_index()

# Now, aggregated_df contains one row per caseID,
# with 'genename' and 'fold_c_value' columns containing lists of values.

aggregated_ase_rows

,caseID,ensembl_genes,foldc_adjust_new
0,TCGA-2A-A8VO,"[ENSG00000091436, ENSG00000131848, ENSG0000019...","[0.18942017743179512, 0.6828771359250264, 1.23..."
1,TCGA-2A-A8VT,"[ENSG00000159840, ENSG00000174442, ENSG0000016...","[0.6213034048266602, 1.6194680584463998, 1.443..."
2,TCGA-2A-A8VV,"[ENSG00000074755, ENSG00000159840, ENSG0000015...","[1.09120612144389, 1.1662320926778185, 0.83460..."
3,TCGA-2A-A8VX,"[ENSG00000091436, ENSG00000159840, ENSG0000012...","[3.0323215295822448, 0.6323230667045006, 0.769..."
4,TCGA-2A-A8W1,"[ENSG00000074755, ENSG00000162378, ENSG0000007...","[0.9524830479450136, 1.8592842071080753, 0.944..."
...,...,...,...
490,TCGA-ZG-A9M4,"[ENSG00000074755, ENSG00000162415, ENSG0000014...","[0.6113333830379262, 0.516567508774503, 0.4334..."
491,TCGA-ZG-A9MC,"[ENSG00000036549, ENSG00000070476, ENSG0000017...","[3.2966079478793766, 0.34299176406443294, 1.88..."
492,TCGA-ZG-A9N3,"[ENSG00000074755, ENSG00000132003, ENSG0000013...","[0.5179727310287839, 0.22299939514922978, 6.39..."
493,TCGA-ZG-A9ND,"[ENSG00000074755, ENSG00000070476, ENSG0000012...","[4.0927262289111255, 1.515322580745505, 1.5944..."


In [17]:
aggregated_ase_rows['caseID'] = aggregated_ase_rows['caseID'].astype(str)

# Apply the mapping
aggregated_ase_rows['target'] = aggregated_ase_rows['caseID'].map(caseID_vectors)

# Now, aggregated_df has a new column 'target' with binary vectors for each caseID
aggregated_ase_rows

,caseID,ensembl_genes,foldc_adjust_new,target
0,TCGA-2A-A8VO,"[ENSG00000091436, ENSG00000131848, ENSG0000019...","[0.18942017743179512, 0.6828771359250264, 1.23...","[0, 0, 0, 0, 0]"
1,TCGA-2A-A8VT,"[ENSG00000159840, ENSG00000174442, ENSG0000016...","[0.6213034048266602, 1.6194680584463998, 1.443...","[0, 0, 0, 1, 0]"
2,TCGA-2A-A8VV,"[ENSG00000074755, ENSG00000159840, ENSG0000015...","[1.09120612144389, 1.1662320926778185, 0.83460...","[0, 0, 0, 1, 0]"
3,TCGA-2A-A8VX,"[ENSG00000091436, ENSG00000159840, ENSG0000012...","[3.0323215295822448, 0.6323230667045006, 0.769...","[0, 0, 0, 0, 0]"
4,TCGA-2A-A8W1,"[ENSG00000074755, ENSG00000162378, ENSG0000007...","[0.9524830479450136, 1.8592842071080753, 0.944...","[0, 0, 0, 0, 0]"
...,...,...,...,...
490,TCGA-ZG-A9M4,"[ENSG00000074755, ENSG00000162415, ENSG0000014...","[0.6113333830379262, 0.516567508774503, 0.4334...","[0, 0, 0, 0, 0]"
491,TCGA-ZG-A9MC,"[ENSG00000036549, ENSG00000070476, ENSG0000017...","[3.2966079478793766, 0.34299176406443294, 1.88...","[0, 0, 0, 1, 1]"
492,TCGA-ZG-A9N3,"[ENSG00000074755, ENSG00000132003, ENSG0000013...","[0.5179727310287839, 0.22299939514922978, 6.39...","[0, 0, 0, 1, 1]"
493,TCGA-ZG-A9ND,"[ENSG00000074755, ENSG00000070476, ENSG0000012...","[4.0927262289111255, 1.515322580745505, 1.5944...","[0, 0, 1, 0, 0]"


In [18]:
def rank_genes(gene_vector, gene_tokens):
    """
    Rank gene expression vector.
    """
    # sort by median-scaled gene values
    sorted_indices = np.argsort(-gene_vector)
    return gene_tokens[sorted_indices]


def tokenize_cell(gene_vector, gene_tokens):
    """
    Convert normalized gene expression vector to tokenized rank value encoding.
    """
    # create array of gene vector with token indices
    # mask undetected genes
    nonzero_mask = np.nonzero(gene_vector)[0]
    # rank by median-scaled gene values
    return rank_genes(gene_vector[nonzero_mask], gene_tokens[nonzero_mask])

In [19]:
# Step 1: Convert ensembl_genes to numerical indices
aggregated_ase_rows['gene_indices'] = aggregated_ase_rows['ensembl_genes'].apply(lambda genes: [gene_token_dict[gene] for gene in genes if gene in gene_token_dict])

In [20]:
def compare_gene_lengths(df):
    # Initialize an empty list to store the results of comparisons
    discrepancies = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the lengths of gene_indices and ensembl_genes for the current row
        len_gene_indices = len(row['gene_indices'])
        len_ensembl_genes = len(row['ensembl_genes'])

        if len_gene_indices != len_ensembl_genes:
            discrepancies.append((index, len_ensembl_genes, len_gene_indices))
            print(f"Row {index}: Length of ensembl_genes is {len_ensembl_genes}, but length of gene_indices is {len_gene_indices}.")

    return discrepancies

discrepancies = compare_gene_lengths(aggregated_ase_rows)

# check if there were any discrepancies
if discrepancies:
    print("There are discrepancies in the lengths.")
else:
    print("All lengths match.")


All lengths match.


In [21]:
# Step 2: Apply tokenize_cell function to each row
aggregated_ase_rows['tokenized_genes'] = aggregated_ase_rows.apply(lambda row: tokenize_cell(np.array(row['foldc_adjust_new']), np.array(row['gene_indices'])), axis=1)

#Collate and Dataset

In [22]:
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class ASEDataset(Dataset):
    def __init__(self, dataframe):
        """
        Initializes the dataset.
        :param dataframe: A pandas DataFrame containing the data.
        """
        self.dataframe = dataframe

    def __len__(self):
        """
        Returns the size of the dataset.
        """
        return len(self.dataframe)

    def __getitem__(self, idx):
        """
        Retrieves a single data point from the dataset.
        """
        row = self.dataframe.iloc[idx]
        gene_tokens = np.array(row['tokenized_genes'])
        fold_change = np.array(row['foldc_adjust_new'])
        target = np.array(row['target'], dtype=np.float32)

        # Return gene_tokens and target as the data dictionary
        return {'gene_tokens': gene_tokens, 'fold_change': fold_change, 'target': target}

In [23]:
import torch

def collate_fn(batch, max_len=512, pad_token=0):
    batch_inputs, batch_targets = [], []

    for item in batch:
        # Extract gene_tokens and target from the current item
        gene_tokens = item['gene_tokens']
        target = item['target']

        # Truncate the gene_tokens to max_len if necessary
        if len(gene_tokens) > max_len:
            gene_tokens = gene_tokens[:max_len]

        # Pad gene_tokens to ensure they are all the same length
        if len(gene_tokens) < max_len:
            padding_length = max_len - len(gene_tokens)
            gene_tokens.extend([pad_token] * padding_length)

        # Append the processed gene_tokens and target to our batch lists
        batch_inputs.append(gene_tokens)
        batch_targets.append(target)

    # Convert lists to PyTorch tensors for model processing
    batch_inputs_tensor = torch.tensor(batch_inputs, dtype=torch.long)
    batch_targets_tensor = torch.tensor(batch_targets, dtype=torch.float)  # Use float for multilabel classification

    return batch_inputs_tensor, batch_targets_tensor


In [24]:
from sklearn.model_selection import train_test_split

# Perform the split
train_df, test_df = train_test_split(aggregated_ase_rows, test_size=0.2, random_state=42)

# Create training and testing datasets
train_dataset = ASEDataset(train_df)
test_dataset = ASEDataset(test_df)

In [25]:
from torch.utils.data import DataLoader

# Define batch size and collate_fn (as previously described)
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Model

In [26]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("ctheodoris/Geneformer")
base_model = AutoModelForMaskedLM.from_pretrained("ctheodoris/Geneformer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/41.2M [00:00<?, ?B/s]

In [27]:
import torch
from torch import nn
from transformers import BertModel

class ASEGeneformer(nn.Module):
    def __init__(self, base_model, num_labels):
        super(ASEGeneformer, self).__init__()
        self.num_labels = num_labels

        # Initialize the BERT model
        self.base_model = base_model.bert

        # Classification head
        self.classification_head = nn.Linear(256, num_labels)

        # Activation function to get probabilities for each label
        self.activation = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        # Get the outputs from BERT model
        outputs = self.base_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        # Extract the last hidden state of the [CLS] token
        cls_output = outputs.last_hidden_state[:, 0, :]

        # Pass through the classification head
        logits = self.classification_head(cls_output)

        # Apply activation function to get probabilities
        probs = self.activation(logits)

        return probs


In [28]:
ase_model = ASEGeneformer(base_model, 5)

In [29]:
ase_model

ASEGeneformer(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25426, 256, padding_idx=0)
      (position_embeddings): Embedding(2048, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.02, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementw

In [31]:
for batch in train_loader:
    inputs, targets = batch

    output = ase_model(inputs)
    print(output)
    break

tensor([[0.5178, 0.3830, 0.4731, 0.6388, 0.4970],
        [0.5428, 0.3916, 0.5368, 0.5273, 0.5244],
        [0.4839, 0.4881, 0.5173, 0.5431, 0.5688],
        [0.5479, 0.4625, 0.5100, 0.6415, 0.5027],
        [0.4760, 0.3056, 0.4844, 0.5763, 0.5651],
        [0.5118, 0.5531, 0.5006, 0.6819, 0.4932],
        [0.3519, 0.4135, 0.5451, 0.5446, 0.3402],
        [0.3893, 0.3813, 0.5168, 0.5769, 0.4495],
        [0.6938, 0.3975, 0.4976, 0.6454, 0.5527],
        [0.5491, 0.3850, 0.3718, 0.6935, 0.5286],
        [0.3993, 0.4024, 0.5520, 0.6118, 0.4746],
        [0.6034, 0.3868, 0.6042, 0.6989, 0.4748],
        [0.4041, 0.3959, 0.5709, 0.5728, 0.5954],
        [0.4523, 0.4149, 0.5036, 0.6631, 0.5452],
        [0.5139, 0.3568, 0.4806, 0.6751, 0.4577],
        [0.4439, 0.3613, 0.5520, 0.5660, 0.4890],
        [0.4236, 0.4074, 0.5673, 0.5819, 0.5470],
        [0.4794, 0.4911, 0.5189, 0.6870, 0.5444],
        [0.4726, 0.3784, 0.5644, 0.4709, 0.6492],
        [0.3604, 0.3960, 0.6090, 0.5421, 0.5257],


#Training

In [42]:
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
import torch

def evaluate_metrics(y_true, y_pred):
    """
    Computes evaluation metrics for multilabel classification.
    """
    # Convert predictions to binary output based on a threshold of 0.5
    y_pred_binary = y_pred > 0.5
    y_pred_binary = y_pred_binary.cpu().numpy()
    y_true = y_true.cpu().numpy()

    metrics = {
        'F1 Score (Macro)': f1_score(y_true, y_pred_binary, average='macro', zero_division=1),
        'Precision (Macro)': precision_score(y_true, y_pred_binary, average='macro', zero_division=1),
        'Recall (Macro)': recall_score(y_true, y_pred_binary, average='macro', zero_division=1),
        'Hamming Loss': hamming_loss(y_true, y_pred_binary)
    }

    return metrics

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)  # Assume outputs are probabilities
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            all_preds.append(outputs.detach())
            all_targets.append(targets.detach())

    # Concatenate all tensors in the list to form a single tensor
    all_preds_tensor = torch.cat(all_preds, dim=0)
    all_targets_tensor = torch.cat(all_targets, dim=0)

    # Calculate metrics
    metrics = evaluate_metrics(all_targets_tensor, all_preds_tensor)

    return total_loss / len(test_loader), metrics


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ase_model.to(device)
optimizer = torch.optim.Adam(ase_model.parameters(), lr=1e-5)
criterion = torch.nn.BCELoss()

# Training and evaluation loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(ase_model, train_loader, optimizer, criterion, device)
    test_loss, metrics = evaluate(ase_model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
    for metric_name, metric_value in metrics.items():
        print(f"{metric_name}: {metric_value:.4f}")
        print("")


Epoch 1, Train Loss: 0.0600, Test Loss: 0.3945
F1 Score (Macro): 0.4754

Precision (Macro): 0.6897

Recall (Macro): 0.4650

Hamming Loss: 0.1253

Epoch 2, Train Loss: 0.0561, Test Loss: 0.3919
F1 Score (Macro): 0.4765

Precision (Macro): 0.6929

Recall (Macro): 0.4650

Hamming Loss: 0.1232

Epoch 3, Train Loss: 0.0531, Test Loss: 0.3946
F1 Score (Macro): 0.4765

Precision (Macro): 0.6929

Recall (Macro): 0.4650

Hamming Loss: 0.1232

Epoch 4, Train Loss: 0.0521, Test Loss: 0.3899
F1 Score (Macro): 0.4738

Precision (Macro): 0.6960

Recall (Macro): 0.4600

Hamming Loss: 0.1212

Epoch 5, Train Loss: 0.0499, Test Loss: 0.4005
F1 Score (Macro): 0.4812

Precision (Macro): 0.6966

Recall (Macro): 0.4700

Hamming Loss: 0.1212

Epoch 6, Train Loss: 0.0478, Test Loss: 0.3947
F1 Score (Macro): 0.4776

Precision (Macro): 0.6963

Recall (Macro): 0.4650

Hamming Loss: 0.1212

Epoch 7, Train Loss: 0.0450, Test Loss: 0.4099
F1 Score (Macro): 0.4857

Precision (Macro): 0.7000

Recall (Macro): 0.4750



In [39]:
for batch in train_loader:
    inputs, targets = batch

    inputs, targets = inputs.to(device), targets.to(device)

    output = ase_model(inputs)
    y_pred = output > 0.5
    print(output)
    print(y_pred)
    print(targets)
    break

tensor([[0.0032, 0.0073, 0.0099, 0.5665, 0.1915],
        [0.0049, 0.0053, 0.0179, 0.6104, 0.2346],
        [0.0038, 0.0073, 0.0109, 0.3469, 0.1495],
        [0.0048, 0.0057, 0.0121, 0.2914, 0.1434],
        [0.0031, 0.0069, 0.0134, 0.4796, 0.2394],
        [0.0039, 0.0060, 0.0160, 0.4862, 0.1640],
        [0.0051, 0.0065, 0.0141, 0.5114, 0.2072],
        [0.0045, 0.0074, 0.0193, 0.5589, 0.2450],
        [0.0050, 0.0063, 0.0257, 0.2516, 0.1808],
        [0.0035, 0.0063, 0.0161, 0.4531, 0.2124],
        [0.0043, 0.0084, 0.0169, 0.3877, 0.1430],
        [0.0033, 0.0059, 0.0129, 0.4521, 0.1614],
        [0.0043, 0.0068, 0.0140, 0.3904, 0.2003],
        [0.0041, 0.0075, 0.0173, 0.6596, 0.3391],
        [0.0039, 0.0062, 0.0138, 0.4082, 0.2117],
        [0.0055, 0.0103, 0.0153, 0.6723, 0.2651],
        [0.0050, 0.0058, 0.0131, 0.6534, 0.1906],
        [0.0043, 0.0090, 0.0132, 0.4915, 0.2148],
        [0.0044, 0.0069, 0.0165, 0.6260, 0.2982],
        [0.0040, 0.0071, 0.0118, 0.3793, 0.1688],


In [45]:
len(train_dataset)

396

In [46]:
len(test_dataset)

99